# SynthDet Evaluation

In this notebook, we provide the FasterRCNN estimator performance and visualization for predictions on the test split of the [UnityGroceries-Real](https://github.com/Unity-Technologies/SynthDet/blob/master/docs/UnityGroceriesReal.md) data. We support performance metrics (mAP, mAP@IOU50, mAR@100) as well as bounding box predictions rendered on the original image. This would provide a better understanding of the given estimator.

<!-- You can use this notebook by the following steps:
- Specify the model path. Then, the notebook would load the checkpoints into a `FasterRCNN` estimator. The `FasterRCNN` can provide model predictions.
- Provide the model performance metrics.
- Can either specify or randomly select some cases for the visualization. -->


## Settings

- Point `data_root` below where you want to download the dataset. If you run this dataset inside a docker container, make sure you point this path to the directory where the external volume is mounted for data storage. 

- Specify a pre-trained `checkpoint_file` that you want to use. You can use one of the estimators that we provided. Optionally, you can use estimators you have trained using our pre-compiled Kubeflow pipeline (Please make sure the GCS credential is setup correctly).

In [1]:
# Specify data_root
data_path = "/Users/bowen.li/data/coco_cat"
checkpoint_file = "gs://thea-test/checkpoints/bowen/GNXBzQ0/run6/FasterRCNN.ep5.estimator"

## Download UnityGroceries-Real Dataset

This cell will download the public UnityGroceries-Real dataset to the location specified by data_path. You only need to run this cell once and assuming the dataset exists for the subsequence of the notebook execution. 

The [downloader](https://datasetinsights.readthedocs.io/en/latest/datasetinsights.io.html#datasetinsights.io.loader.create_loader) instantiates the dataset downloader after finding it with the source-uri provided.

In [3]:
from datasetinsights.io import create_downloader

coco_cat_img_source_uri = "gs://thea-dev/data/coco_cat/train2017.zip"
downloader = create_downloader(source_uri=coco_cat_img_source_uri)
downloader.download(source_uri=coco_cat_img_source_uri, output=data_path)

/Users/bowen.li/miniconda3/envs/bowen/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/bowen.li/miniconda3/envs/bowen/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For mor

In [4]:
coco_cat_ann_source_uri = "gs://thea-dev/data/coco_cat/annotations_trainval2017.zip"
downloader = create_downloader(source_uri=coco_cat_ann_source_uri)
downloader.download(source_uri=coco_cat_ann_source_uri, output=data_path)

/Users/bowen.li/miniconda3/envs/bowen/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/bowen.li/miniconda3/envs/bowen/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For mor

Load UnityGroceries-Real [test](https://github.com/Unity-Technologies/SynthDet/blob/master/docs/Readme.md) split which has 254 images.

In [2]:
from datasetinsights.datasets import Dataset

test_dataset = Dataset.create("CocoDetection", data_path=data_path, split="train")

loading annotations into memory...
Done (t=0.10s)
creating index...
index created!


## Load Estimator
An [estimator](https://datasetinsights.readthedocs.io/en/latest/datasetinsights.estimators.html#datasetinsights.estimators.base.Estimator) is a class of one modeling operation. It includes:

1. input data and output data transformations (e.g. input image cropping, remove unused output labels…) when applicable. 
2. neural network graph (model) for either PyTorch or TensorFlow. 
3. procedures to execute model training and evaluation. <br>

This cell will load an estimator specified in the variable name `checkpoint_file`.

In [4]:
from yacs.config import CfgNode as CN
from datasetinsights.estimators import create_estimator
    
config_yaml = """
    estimator: FasterRCNN
    backbone: resnet50
    num_classes: 2
    task: object_detection
    test:
      batch_size: 8
      dataset:
        name: CocoDetection
        args:
          version: v3
          split: test
    metrics:
      mean-average-precision:
        name: MeanAveragePrecisionAverageOverIOU
      mean-average-precision-iou50:
        name: MeanAveragePrecisionIOU50
      mean-average-recall:
        name: MeanAverageRecallAverageOverIOU
      average-precision-iou50:
        name: AveragePrecisionIOU50
    pretrained: False
    pretrained_backbone: True
    synchronize_metrics: True
"""
config = CN.load_cfg(config_yaml)
estimator = create_estimator(
    name=config.estimator,
    config=config,
    checkpoint_file=checkpoint_file,
)

/Users/bowen.li/miniconda3/envs/bowen/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/bowen.li/miniconda3/envs/bowen/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/Users/bowen.li/miniconda3/envs/bowen/lib/python3.7

## Prediction Visualization

In order to improve visual inspection, we have color-coded bounding boxes predictions based on IOU value between prediction and ground truth bounding boxes. It is considered true positive if `IOU >= 0.5`. We only visualize prediction bounding box with `score >= 0.5`. 

- <font color='green'>Green boxes</font>: If the predicted bounding box can be matched to a ground truth bounding box. <br>
- <font color='red'>Red boxes</font>: If the predicted bounding box can't be matched to a ground truth bounding box. <br>

In [5]:
from datasetinsights.stats.visualization import match_boxes, plot_bboxes, grid_plot

def visualize_predictions(index=0):
    """ Plot ground truth and prediction for one image.
    
    This method would plot two images: the ground truth is on the left;
    the prediction from the loaded estimator is on the right.
    """
    estimator.model.eval()
    pil_image, gt_bboxes = test_dataset[index]
    pred_bboxes = estimator.predict(pil_image, box_score_thresh=0.5)
    colors = match_boxes(pred_bboxes, gt_bboxes)
    gt_plot = plot_bboxes(pil_image, gt_bboxes, test_dataset.label_mappings)
    pred_plot = plot_bboxes(pil_image, pred_bboxes, test_dataset.label_mappings, colors)
    
    titles = [
        f"ground truth for image index: {index}",
        f"prediction for image index: {index}",
    ]
    grid_plot([[gt_plot, pred_plot]], figsize=(7, 10), img_type="rgb", titles=titles)

/Users/bowen.li/miniconda3/envs/bowen/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
from ipywidgets import interact

# switch to evaluation mode.
estimator_eval = estimator.model.eval()
# Please select an index of a image.
interact(visualize_predictions, index=list(range(len(test_dataset))))

interactive(children=(Dropdown(description='index', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,…

<function __main__.visualize_predictions(index=0)>

## Estimator Performance
This section calculates estimator performance on the UnityGroceries-Real Dataset test split. We report the following three evaluation metrics that are commonly used for object detection task:
- [mAP](https://datasetinsights.readthedocs.io/en/latest/datasetinsights.evaluation_metrics.html#datasetinsights.evaluation_metrics.average_precision_2d.MeanAveragePrecisionAverageOverIOU): Average Precision average over all the labels and IOU thresholds = 0.5:0.95:0.05
- [mAPIOU50](https://datasetinsights.readthedocs.io/en/latest/datasetinsights.evaluation_metrics.html#datasetinsights.evaluation_metrics.average_precision_2d.MeanAveragePrecisionIOU50): Mean Average Precision at IOU=50%.
- [mAR](https://datasetinsights.readthedocs.io/en/latest/datasetinsights.evaluation_metrics.html#datasetinsights.evaluation_metrics.average_recall_2d.MeanAverageRecallAverageOverIOU): Average Recall average over all the labels and IOU thresholds = 0.5:0.95:0.05

The next cell will take ~1 hour if you run model precision locally without GPU support. This will make predictions on the whole test set of 254 images. You can reduce computation time significantly on GPU with CUDA support.

In [ ]:
# This is expected to take a while
estimator.evaluate(data_path)

In [ ]:
import pandas as pd

metric_names = ["mAP", "mAPIOU50", "mAR"]
metrics = estimator.metrics
df = pd.DataFrame({
    "Value": [metrics[name].compute() for name in metric_names],
})
df.index = metric_names
df